# Lab 7: Text Classification with SVM

In [1]:
from sklearn.datasets import load_files
movie_reviews_data_folder = 'movie_reviews/txt_sentoken'
dataset = load_files(movie_reviews_data_folder, shuffle=False)

Labels

In [2]:
dataset.target_names

['neg', 'pos']

Training data

In [3]:
dataset.filenames

array(['movie_reviews/txt_sentoken\\neg\\cv000_29416.txt',
       'movie_reviews/txt_sentoken\\neg\\cv001_19502.txt',
       'movie_reviews/txt_sentoken\\neg\\cv002_17424.txt', ...,
       'movie_reviews/txt_sentoken\\pos\\cv997_5046.txt',
       'movie_reviews/txt_sentoken\\pos\\cv998_14111.txt',
       'movie_reviews/txt_sentoken\\pos\\cv999_13106.txt'], 
      dtype='|S46')

In [4]:
len(dataset.data)

2000

Split data into train data and test data

In [6]:
from sklearn.cross_validation import train_test_split
docs_train, docs_test, y_train, y_test = train_test_split(dataset.data, dataset.target, test_size=0.25, random_state=None)

TASK: Build a vectorizer / classifier pipeline that filters out tokens that are too rare or too frequent

In [29]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC, LinearSVC, NuSVC
import numpy as np
pipe = Pipeline([('vect', TfidfVectorizer()), 
                  ('clf', LinearSVC())])
pipe.fit(docs_train,y_train)
predicted = pipe.predict(docs_test)
np.mean(predicted == y_test)

0.85999999999999999

TASK: Build a grid search to find out whether unigrams or bigrams are more useful.

Fit the pipeline on the training set using grid search for the parameters

In [31]:
from sklearn.grid_search import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
              'vect__use_idf': (True, False),
              'clf__C': (1.0, 0.1, 1e-2, 1e-3),
}
gs_clf = GridSearchCV(pipe, parameters, n_jobs=-1)
gs_clf = gs_clf.fit(docs_train, y_train)

In [25]:
gs_clf.best_params_

{'clf__C': 1.0, 'vect__ngram_range': (1, 1), 'vect__use_idf': True}

Print the mean and std for each candidate along with the parameter settings for all the candidates explored by grid search.

Predict the outcome on the testing set and store it in a variable named y_predicted

In [26]:
y_predicted = gs_clf.predict(docs_test)

#### Print the classification report

In [27]:
from sklearn import metrics
print(metrics.classification_report(y_test, y_predicted))

             precision    recall  f1-score   support

          0       0.87      0.84      0.86       250
          1       0.85      0.88      0.86       250

avg / total       0.86      0.86      0.86       500



#### Print and plot the confusion matrix

In [28]:
cm = metrics.confusion_matrix(y_test, y_predicted)
print(cm)

[[211  39]
 [ 31 219]]
